<a href="https://colab.research.google.com/github/VIPlearner/csv-files/blob/main/KaggleTitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
#installing relevant libraries
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [249]:
#importing data from my github profile
train_df=pd.read_csv("https://raw.githubusercontent.com/VIPlearner/csv-files/main/train.csv", index_col = 'PassengerId')
test_df = pd.read_csv("https://raw.githubusercontent.com/VIPlearner/csv-files/main/test.csv", index_col = 'PassengerId')
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [250]:
train_df['train'] = 1
test_df['train'] = 0

In [251]:
all_df = pd.concat([train_df.drop('Survived', axis = 1), test_df])
target = train_df['Survived']
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    1309 non-null   int64  
 1   Name      1309 non-null   object 
 2   Sex       1309 non-null   object 
 3   Age       1046 non-null   float64
 4   SibSp     1309 non-null   int64  
 5   Parch     1309 non-null   int64  
 6   Ticket    1309 non-null   object 
 7   Fare      1308 non-null   float64
 8   Cabin     295 non-null    object 
 9   Embarked  1307 non-null   object 
 10  train     1309 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [252]:
all_df['Age'] = SimpleImputer().fit_transform(all_df[['Age']])
# all_df['Fare'] = SimpleImputer().fit_transform(all_df[['Fare']])
all_df['Embarked'] = SimpleImputer(strategy = 'most_frequent').fit_transform(all_df[['Embarked']])
all_df['Cabin'] = SimpleImputer(strategy = 'most_frequent').fit_transform(all_df[['Cabin']])

all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    1309 non-null   int64  
 1   Name      1309 non-null   object 
 2   Sex       1309 non-null   object 
 3   Age       1309 non-null   float64
 4   SibSp     1309 non-null   int64  
 5   Parch     1309 non-null   int64  
 6   Ticket    1309 non-null   object 
 7   Fare      1308 non-null   float64
 8   Cabin     1309 non-null   object 
 9   Embarked  1309 non-null   object 
 10  train     1309 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [253]:
#dropping irrelevant features
all_df.drop(['Name'], axis = 1, inplace = True)
# all_df.drop(['Cabin'], axis = 1, inplace = True)
#df.drop(['Model'], axis = 1, inplace = True)
# df.drop(['Distance'], axis = 1, inplace = True)
all_df['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6, 9])

In [254]:
#converting categorical columns to dtype category for the One Hot Encoder
all_df['Pclass'] = all_df.Pclass.astype('category')
all_df['Sex'] = all_df.Sex.astype('category')
all_df['SibSp'] = all_df.SibSp.astype('category')
all_df['Parch'] = all_df.Parch.astype('category')
all_df['Ticket'] = all_df.Ticket.astype('category')
all_df['Cabin'] = all_df.Cabin.astype('category')
all_df['Embarked'] = all_df.Embarked.astype('category')

# df['Colour'] = df.Colour.astype('category')
all_df.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train
PassengerId,,,,,,,,,,
1,3,male,22.0,1,0,A/5 21171,7.2500,C23 C25 C27,S,1
2,1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,C23 C25 C27,S,1
4,1,female,35.0,1,0,113803,53.1000,C123,S,1
5,3,male,35.0,0,0,373450,8.0500,C23 C25 C27,S,1


In [255]:
cat_col =['Pclass', 
          'Sex', 
          'SibSp', 
          'Parch', 
          'Ticket', 
          'Cabin', 
          'Embarked', 
          'Age'
          ]
for col in cat_col:
  all_df = pd.get_dummies(all_df,prefix=[col], columns = [col], drop_first=True)
all_df.head()

,Fare,train,Pclass_2,Pclass_3,Sex_male,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,...,Age_64.0,Age_65.0,Age_66.0,Age_67.0,Age_70.0,Age_70.5,Age_71.0,Age_74.0,Age_76.0,Age_80.0
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,7.2500,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,71.2833,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7.9250,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53.1000,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,8.0500,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [256]:
noncat_col = ['Fare']
for col in noncat_col:
  all_df[col] = StandardScaler().fit_transform(all_df[[col]])
all_df.head()

,Fare,train,Pclass_2,Pclass_3,Sex_male,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,...,Age_64.0,Age_65.0,Age_66.0,Age_67.0,Age_70.0,Age_70.5,Age_71.0,Age_74.0,Age_76.0,Age_80.0
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,-0.503402,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.734222,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.490356,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.382778,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,-0.487940,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [257]:
train_df = all_df[all_df['train'] == 1].drop('train', axis = 1)
test_df = all_df[all_df['train'] == 0].drop('train', axis = 1)

In [258]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target,
                                                    test_size = 0.3,
                                                   random_state = 42)

In [259]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier()

In [260]:
print("model score: %.3f" % model.score(X_test, y_test))
y_pred = model.predict(X_test)
errors = mean_squared_error(y_test, y_pred, squared = False)
print(errors)

model score: 0.780
0.4692006540184522


In [108]:
test_y_pred = model.predict(test_df)

In [109]:
test_pred_df = pd.DataFrame({'PassengerId' : test_df.index, 'Survived': test_y_pred})
test_pred_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [110]:
# converting dataframe to csv file
test_pred_df.to_csv('ninthtry.csv', encoding='utf-8', index=False)

In [111]:
# downloading csv file
from google.colab import files
files.download("ninthtry.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>